In [16]:
import math
import numpy
import itertools
import sympy.ntheory

def ASCIIPad(Mess,Mod):
    K = []
    for letter in Mess:
        K.append(ord(letter))
    L = Mod.ndigits()
    l = len(K)
    y = ZZ(math.floor(L/3))
    count = 0
    padded = []
    buffer = ""
    for numChar in K:
        numChar+=100
        buffer+=str(numChar)
        count+=1
        if count==y:
            padded.append(ZZ(buffer))
            buffer = ""
            count = 0
    if len(buffer)>0:
        padded.append(ZZ(buffer))
    return padded

def ASCIIDepad(Number):
    N = "";
    n = Number.ndigits() % 3;
    if (n > 0):
        print("This is not a padded ASCII string\n");
    else:
        L = [((Number - (Number % (1000^i)))/1000^i)%1000 - 100 for i in range(Number.ndigits()/3)];
        for i in range(Number.ndigits()/3):
            N = chr(L[i]) + N;
    return(N);

def TonSh (a, Prime):
    if kronecker(a, Prime) == -1:
        pass
        print ("{0} does not have a square root mod {1}".format(a, Prime))
        return None
    elif Prime % 4 == 3:
        x = power_mod(ZZ(a), ZZ((Prime+1)/4),ZZ(Prime))
        return(x)
    else:
        #################################################################################
        # Determine e so that Prime-1 = (2^e)*q for some odd number q                   # 
        #################################################################################

        e = ZZ(0)
        q = ZZ(Prime - 1)
        for j in itertools.count(1):
            if q % 2 == 0:
                e = ZZ(e + 1)
                q = ZZ(q / 2)
            else:
                break
        for i in range(1, 101):
            n = i
            if kronecker(n, Prime) == -1:
                break
        z = power_mod(ZZ(n),ZZ(q),ZZ(Prime))
        y = ZZ(z)
        r = ZZ(e)
        x = power_mod(ZZ(a),ZZ( (q-1)/2),ZZ( Prime))
        b = (a * x ** 2) % Prime
        x = (a * x) % Prime
        #for i in range(1, e + 1):
        for i in itertools.count(1):
            if b == 1:
                break
            else:
                for m in itertools.count(1):
                    t = power_mod(ZZ(b), ZZ(2^m) ,  ZZ(Prime))
                    if t == 1:
                        mm = m
                        break
                t = power_mod(ZZ(y), ZZ(2^(r - mm - 1)),ZZ(Prime))
                y = power_mod(ZZ(t), ZZ(2),ZZ(Prime))
                r = mm
                x = (x * t) % Prime
                b = (b * y) % Prime
        return(x)

def ASCIIPad(Mess,Mod):
    K = []
    for letter in Mess:
        K.append(ord(letter))
    L = Mod.ndigits()
    l = len(K)
    y = ZZ(math.floor(L/3))
    count = 0
    padded = []
    buffer = ""
    for numChar in K:
        numChar+=100
        buffer+=str(numChar)
        count+=1
        if count==y:
            padded.append(ZZ(buffer))
            buffer = ""
            count = 0
    if len(buffer)>0:
        padded.append(ZZ(buffer))
    return padded

def ASCIIDepad(Number):
    N = "";
    n = Number.ndigits() % 3;
    if (n > 0):
        print("This is not a padded ASCII string\n");
    else:
        L = [((Number - (Number % (1000^i)))/1000^i)%1000 - 100 for i in range(Number.ndigits()/3)];
        for i in range(Number.ndigits()/3):
            N = chr(L[i]) + N;
    return(N);

def ECEmbed (Message, gp, tolerance):
    p = ZZ(math.floor(gp[2] / (tolerance + 1)))
    M = ASCIIPad(Message, p)
    packets = len(M)
    pointlist = [0]*packets
    for j in range(0, packets ):
        N = M[j]
        pointlist[j] = ECSearch(tolerance * N, tolerance * (N + 1) - 1, gp)
    return(pointlist)

def ECUnembed (pointlist, tolerance):
    k = len(pointlist)
    paddedasciilist=[0]*k
    for j in range(0, k ):
        pointlist[j][0]=ZZ(pointlist[j][0])
        toType = ZZ(QQ((pointlist[j][0])/tolerance).floor())
        paddedasciilist[j] = ((pointlist[j][0])/tolerance).floor()
    returnStr = ""
    for paddedItem in paddedasciilist:
        buffer = ASCIIDepad(paddedItem)
        returnStr+= buffer
    return returnStr

def ECSearch (lowerbound, upperbound, Group):
    p = Group[2]
    a = Group[0]
    b = Group[1]
    if (4 * a ** 3 + 27 * b ** 2) % p == 0:
        print ("This is not an elliptic curve.")
    else:
        for j in itertools.count(lowerbound):
            if j==lowerbound-1 or j>upperbound or j>upperbound:
                return "not found"
            j=j%p
            if kronecker(j ** 3 + a * j + b, p) == 1:
                x = (j ** 3 + a * j + b) % p
                var('z')
                L=TonSh(x,p)
                y = L
                return([j,y])

def ECAdd (Point1, Point2, Group):
    a = Group[0]
    b = Group[1]
    p = Group[2]
    if(Point1!=[]):
        x1 = Point1[0]
        y1 = Point1[1]
    if(Point2!=[]):
        x2 = Point2[0]
        y2 = Point2[1]
    if (4 * a ** 3 + 27 * b ** 2) % p == 0:
        print ("This is not an elliptic curve. ")
    elif Point1 != [] and y1 ** 2 % p != (x1 ** 3 + a * x1 + b) % p:
        print ("Point 1 is not on the elliptic curve. \n")
    elif Point2 != [] and y2 ** 2 % p != (x2 ** 3 + a * x2 + b) % p:
        print ("Point 2 is not on the elliptic curve. \n")
    else:
        if Point1 == []:
            R = Point2
        elif Point2 == []:
            R = Point1
        else:
            if x1 == x2 and 0 == (y1 + y2) % p:
                R = []
            elif x1 == x2 and y1 == y2:
                R = ECDouble(Point1, Group)
            else:
                s = ((y1 - y2) / (x1 - x2)) % p
                x = (s ** 2 - x1 - x2) % p
                y = (s * (x1 - x) - y1) % p
                R = [x,y]
    return(R)

def ECDouble (Point, Group):
    a = Group[0]
    b = Group[1]
    p = Group[2]
    if Point != []:
        x1 = Point[0]
        y1 = Point[1]
        if (4 * a ** 3 + 27 * b ** 2) % p == 0:
            print ("This is not an elliptic curve. ")
        elif Point != [] and y1 ** 2 % p != (x1 ** 3 + a * x1 + b) % p:
            print ("The point to double is not on the elliptic curve. ")
        elif y1 == 0:
            R = []
        else:
            s = mod((3 * x1 ** 2 + a) / y1 / 2, p)
            x = (s ** 2 - 2 * x1) % p
            y = (s * (x1 - x) - y1) % p
            R = [x,y]
    else:
        R = []
    return(R)

def ECInverse (Point, Group):
    if Point == []:
        return(Point)
    else:
        p = Group[2]
        x = Point[0]
        y = Point[1]
        return([x,(p - y) % p])

def ECTimes (Point, scalar, Group):
    ECIdentity=[]
    if Point == ECIdentity or scalar == 0:
        return(ECIdentity)
    else:
        E=EllipticCurve(GF(Group[2]),[Group[0],Group[1]])
        EPoint = E(Point[0],Point[1])
        cgret = scalar*EPoint
        if(cgret[0]==0 and cgret[1]==1 and cgret[2]==0):
            return ECIdentity
        return([cgret[0],cgret[1]])

def findptOrder(point,group):
    E = EllipticCurve(GF(group[2]),[group[0],group[1]])
    Ep = E(point[0],point[1])
    return Ep.additive_order()

def ECEncrypt(M,g,r,b,G,t):
    m=ECEmbed(M,G,t)
    C = []
    for i in range(len(m)):
        c1 = ECTimes(b,r,G)
        c1 = ECAdd(c1,m[i],G)
        c2 = ECTimes(g,r,G)
        C = C + [[c1,c2]]
    return C

def ECDecryption(C,x,G,T):
    M = [0]*len(C)
    for i in range(len(C)):
        temp = ECTimes(C[i][1],x,G)
        temp = ECAdd(C[i][0],ECInverse(temp,G),G)
        M[i] = temp
    P = ECUnembed(M,T)
    return P



# Initialize the parameters to define an elliptic curve group
A = 453247983724987324739287983275982375923
B = 0
p = next_prime(48756928743475984375947598473598437598473597439857498375948375984375984375983475)
G = [A, B, p]  # The elliptic curve group is defined by parameters A, B, and prime p
print("The elliptic curve group is G =", G,"\n")

# Calculate the order of the elliptic curve group G
E = EllipticCurve(GF(p), [A, B])
Gp = E.order()  # The total number of points in the group G
print("The order of the elliptic curve group is", Gp,"\n")

# Select a generator point g on the elliptic curve group G
g = ECSearch(78657643, 894579475973497, G)
print("The base point is g =", g,"\n")

# Determine the order of the base point g in the group G
gorder = findptOrder(g, G)  # The cyclic subgroup order generated by g
print("The order of the base point g is", gorder,"\n")

# Define a private key for ECC
x = 3267452870001
print("The private key is x =", x,"\n")

# Calculate the public key by multiplying the base point g by the private key x
b = ECTimes(g, x, G)
print("The public elliptic curve point (public key) is b =", b,"\n")

# Encrypt a plaintext message using the elliptic curve
message = "hello" #why does the example have such a restrictive EC
m1 = ASCIIPad(message, (p / 48).floor())  # Pad the message for encryption
m = m1[0]  # Use the first block of the padded message for demonstration

# Choose a random number r such that gcd(r, order of g) = 1
r = 892173201

# Generate a point R on the elliptic curve using the base point g and the random number r
R = ECTimes(g, r, G)
Y = R[0]  # The first coordinate of the point R represents part of the signature

# Compute the signature value S using the padded message, private key, random number r, and the point R
m = ZZ(m)
x = ZZ(x)
Y = ZZ(Y)
r = ZZ(r)
S = mod((m + x * Y) / r, gorder)

print("The elliptic curve signature is the pair Y =", Y, ", S =", S,"\n")

# Verify the signature by computing U1 and U2 and then adding the resulting points
U1 = ZZ(mod(m / S, gorder))
U2 = ZZ(mod(Y / S, gorder))
V = ECAdd(ECTimes(g, U1, G), ECTimes(b, U2, G), G)
if V == R:
    print("Valid Signature")
else:
    print("Invalid Signature")

    
###### Choose a plaintext
Plaintext = "According to all known laws of aviation, there is no way a bee should be able to fly."

###### Choose a random number
r = 234563456745678
##### Define the tollerance parameter
T=50
###### Use the function ECEncrypt to encrypt the plaintext
Ciphertext=ECEncrypt(Plaintext,g,r,b,G,T)
print("The cipherext is the following array of pairs of elliptic curve points\n", Ciphertext)

T=50
ECDecryption(Ciphertext,x,G,T)


The key was dropped by Alice
False 2670593282590084201080289828563613325182025947593 1985977078462597337627134881095450804747548473623
False 564494786998998224486536343651104367885400531139 1985977078462597337627134881095450804747548473623


In [12]:
import math
import numpy
import itertools
import sympy.ntheory

def ASCIIPad(Mess,Mod):
    K = []
    for letter in Mess:
        K.append(ord(letter))
    L = Mod.ndigits()
    l = len(K)
    y = ZZ(math.floor(L/3))
    count = 0
    padded = []
    buffer = ""
    for numChar in K:
        numChar+=100
        buffer+=str(numChar)
        count+=1
        if count==y:
            padded.append(ZZ(buffer))
            buffer = ""
            count = 0
    if len(buffer)>0:
        padded.append(ZZ(buffer))
    return padded

def ASCIIDepad(Number):
    N = "";
    n = Number.ndigits() % 3;
    if (n > 0):
        print("This is not a padded ASCII string\n");
    else:
        L = [((Number - (Number % (1000^i)))/1000^i)%1000 - 100 for i in range(Number.ndigits()/3)];
        for i in range(Number.ndigits()/3):
            N = chr(L[i]) + N;
    return(N);

def TonSh (a, Prime):
    if kronecker(a, Prime) == -1:
        pass
        print ("{0} does not have a square root mod {1}".format(a, Prime))
        return None
    elif Prime % 4 == 3:
        x = power_mod(ZZ(a), ZZ((Prime+1)/4),ZZ(Prime))
        return(x)
    else:
        #################################################################################
        # Determine e so that Prime-1 = (2^e)*q for some odd number q                   # 
        #################################################################################

        e = ZZ(0)
        q = ZZ(Prime - 1)
        for j in itertools.count(1):
            if q % 2 == 0:
                e = ZZ(e + 1)
                q = ZZ(q / 2)
            else:
                break
        for i in range(1, 101):
            n = i
            if kronecker(n, Prime) == -1:
                break
        z = power_mod(ZZ(n),ZZ(q),ZZ(Prime))
        y = ZZ(z)
        r = ZZ(e)
        x = power_mod(ZZ(a),ZZ( (q-1)/2),ZZ( Prime))
        b = (a * x ** 2) % Prime
        x = (a * x) % Prime
        #for i in range(1, e + 1):
        for i in itertools.count(1):
            if b == 1:
                break
            else:
                for m in itertools.count(1):
                    t = power_mod(ZZ(b), ZZ(2^m) ,  ZZ(Prime))
                    if t == 1:
                        mm = m
                        break
                t = power_mod(ZZ(y), ZZ(2^(r - mm - 1)),ZZ(Prime))
                y = power_mod(ZZ(t), ZZ(2),ZZ(Prime))
                r = mm
                x = (x * t) % Prime
                b = (b * y) % Prime
        return(x)

def ASCIIPad(Mess,Mod):
    K = []
    for letter in Mess:
        K.append(ord(letter))
    L = Mod.ndigits()
    l = len(K)
    y = ZZ(math.floor(L/3))
    count = 0
    padded = []
    buffer = ""
    for numChar in K:
        numChar+=100
        buffer+=str(numChar)
        count+=1
        if count==y:
            padded.append(ZZ(buffer))
            buffer = ""
            count = 0
    if len(buffer)>0:
        padded.append(ZZ(buffer))
    return padded

def ASCIIDepad(Number):
    N = "";
    n = Number.ndigits() % 3;
    if (n > 0):
        print("This is not a padded ASCII string\n");
    else:
        L = [((Number - (Number % (1000^i)))/1000^i)%1000 - 100 for i in range(Number.ndigits()/3)];
        for i in range(Number.ndigits()/3):
            N = chr(L[i]) + N;
    return(N);

def ECEmbed (Message, gp, tolerance):
    p = ZZ(math.floor(gp[2] / (tolerance + 1)))
    M = ASCIIPad(Message, p)
    packets = len(M)
    pointlist = [0]*packets
    for j in range(0, packets ):
        N = M[j]
        pointlist[j] = ECSearch(tolerance * N, tolerance * (N + 1) - 1, gp)
    return(pointlist)

def ECUnembed (pointlist, tolerance):
    k = len(pointlist)
    paddedasciilist=[0]*k
    for j in range(0, k ):
        pointlist[j][0]=ZZ(pointlist[j][0])
        toType = ZZ(QQ((pointlist[j][0])/tolerance).floor())
        paddedasciilist[j] = ((pointlist[j][0])/tolerance).floor()
    returnStr = ""
    for paddedItem in paddedasciilist:
        buffer = ASCIIDepad(paddedItem)
        returnStr+= buffer
    return returnStr

def ECSearch (lowerbound, upperbound, Group):
    p = Group[2]
    a = Group[0]
    b = Group[1]
    if (4 * a ** 3 + 27 * b ** 2) % p == 0:
        print ("This is not an elliptic curve.")
    else:
        for j in itertools.count(lowerbound):
            if j==lowerbound-1 or j>upperbound or j>upperbound:
                return "not found"
            j=j%p
            if kronecker(j ** 3 + a * j + b, p) == 1:
                x = (j ** 3 + a * j + b) % p
                var('z')
                L=TonSh(x,p)
                y = L
                return([j,y])

def ECAdd (Point1, Point2, Group):
    a = Group[0]
    b = Group[1]
    p = Group[2]
    if(Point1!=[]):
        x1 = Point1[0]
        y1 = Point1[1]
    if(Point2!=[]):
        x2 = Point2[0]
        y2 = Point2[1]
    if (4 * a ** 3 + 27 * b ** 2) % p == 0:
        print ("This is not an elliptic curve. ")
    elif Point1 != [] and y1 ** 2 % p != (x1 ** 3 + a * x1 + b) % p:
        print ("Point 1 is not on the elliptic curve. \n")
    elif Point2 != [] and y2 ** 2 % p != (x2 ** 3 + a * x2 + b) % p:
        print ("Point 2 is not on the elliptic curve. \n")
    else:
        if Point1 == []:
            R = Point2
        elif Point2 == []:
            R = Point1
        else:
            if x1 == x2 and 0 == (y1 + y2) % p:
                R = []
            elif x1 == x2 and y1 == y2:
                R = ECDouble(Point1, Group)
            else:
                s = ((y1 - y2) / (x1 - x2)) % p
                x = (s ** 2 - x1 - x2) % p
                y = (s * (x1 - x) - y1) % p
                R = [x,y]
    return(R)

def ECDouble (Point, Group):
    a = Group[0]
    b = Group[1]
    p = Group[2]
    if Point != []:
        x1 = Point[0]
        y1 = Point[1]
        if (4 * a ** 3 + 27 * b ** 2) % p == 0:
            print ("This is not an elliptic curve. ")
        elif Point != [] and y1 ** 2 % p != (x1 ** 3 + a * x1 + b) % p:
            print ("The point to double is not on the elliptic curve. ")
        elif y1 == 0:
            R = []
        else:
            s = mod((3 * x1 ** 2 + a) / y1 / 2, p)
            x = (s ** 2 - 2 * x1) % p
            y = (s * (x1 - x) - y1) % p
            R = [x,y]
    else:
        R = []
    return(R)

def ECInverse (Point, Group):
    if Point == []:
        return(Point)
    else:
        p = Group[2]
        x = Point[0]
        y = Point[1]
        return([x,(p - y) % p])

def ECTimes (Point, scalar, Group):
    ECIdentity=[]
    if Point == ECIdentity or scalar == 0:
        return(ECIdentity)
    else:
        E=EllipticCurve(GF(Group[2]),[Group[0],Group[1]])
        EPoint = E(Point[0],Point[1])
        cgret = scalar*EPoint
        if(cgret[0]==0 and cgret[1]==1 and cgret[2]==0):
            return ECIdentity
        return([cgret[0],cgret[1]])

def findptOrder(point,group):
    E = EllipticCurve(GF(group[2]),[group[0],group[1]])
    Ep = E(point[0],point[1])
    return Ep.additive_order()

def ECEncrypt(M,g,r,b,G,t):
    m=ECEmbed(M,G,t)
    C = []
    for i in range(len(m)):
        c1 = ECTimes(b,r,G)
        c1 = ECAdd(c1,m[i],G)
        c2 = ECTimes(g,r,G)
        C = C + [[c1,c2]]
    return C

def ECDecryption(C,x,G,T):
    M = [0]*len(C)
    for i in range(len(C)):
        temp = ECTimes(C[i][1],x,G)
        temp = ECAdd(C[i][0],ECInverse(temp,G),G)
        M[i] = temp
    P = ECUnembed(M,T)
    return P



# Initialize the parameters to define an elliptic curve group
A = 453247983724987324739287983275982375923
B = 0
p = next_prime(48756928743475984375947598473598437598473597439857498375948375984375984375983475)
G = [A, B, p]  # The elliptic curve group is defined by parameters A, B, and prime p
print("The elliptic curve group is G =", G,"\n")

# Calculate the order of the elliptic curve group G
E = EllipticCurve(GF(p), [A, B])
Gp = E.order()  # The total number of points in the group G
print("The order of the elliptic curve group is", Gp,"\n")

# Select a generator point g on the elliptic curve group G
g = ECSearch(78657643, 894579475973497, G)
print("The base point is g =", g,"\n")

# Determine the order of the base point g in the group G
gorder = findptOrder(g, G)  # The cyclic subgroup order generated by g
print("The order of the base point g is", gorder,"\n")

# Define a private key for ECC
x = 3267452870001
print("The private key is x =", x,"\n")

# Calculate the public key by multiplying the base point g by the private key x
b = ECTimes(g, x, G)
print("The public elliptic curve point (public key) is b =", b,"\n")

# Encrypt a plaintext message using the elliptic curve
message = "hello" #why does the example have such a restrictive EC
m1 = ASCIIPad(message, (p / 48).floor())  # Pad the message for encryption
m = m1[0]  # Use the first block of the padded message for demonstration

# Choose a random number r such that gcd(r, order of g) = 1
r = 892173201

# Generate a point R on the elliptic curve using the base point g and the random number r
R = ECTimes(g, r, G)
Y = R[0]  # The first coordinate of the point R represents part of the signature

# Compute the signature value S using the padded message, private key, random number r, and the point R
m = ZZ(m)
x = ZZ(x)
Y = ZZ(Y)
r = ZZ(r)
S = mod((m + x * Y) / r, gorder)

print("The elliptic curve signature is the pair Y =", Y, ", S =", S,"\n")

# Verify the signature by computing U1 and U2 and then adding the resulting points
U1 = ZZ(mod(m / S, gorder))
U2 = ZZ(mod(Y / S, gorder))
V = ECAdd(ECTimes(g, U1, G), ECTimes(b, U2, G), G)
if V == R:
    print("Valid Signature")
else:
    print("Invalid Signature")

    
###### Choose a plaintext
Plaintext = "According to all known laws of aviation, there is no way a bee should be able to fly."

###### Choose a random number
r = 234563456745678
##### Define the tollerance parameter
T=50
###### Use the function ECEncrypt to encrypt the plaintext
Ciphertext=ECEncrypt(Plaintext,g,r,b,G,T)
print("The cipherext is the following array of pairs of elliptic curve points\n", Ciphertext)

T=50
ECDecryption(Ciphertext,x,G,T)
